In [6]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Load the raw data
raw_data = pd.read_csv('/content/raw_data.csv')
incremental_data = pd.read_csv('/content/incremental_data.csv')

# Custom transformations for this specific dataset
def clean_data(df):
    """Apply all transformations to the dataframe"""
    df = df.copy() # Create a copy to avoid SettingWithCopyWarning

    # 1. Handle missing values
    print("\n1. Handling missing values...")
    # Customer name - fill with "Unknown"
    df['customer_name'] = df['customer_name'].fillna('Unknown')

    # Quantity - fill with median by product, then 0 for any remaining NaNs
    quantity_medians = df.groupby('product')['quantity'].median()
    df['quantity'] = df.apply(
        lambda row: quantity_medians[row['product']] if pd.isna(row['quantity']) else row['quantity'],
        axis=1
    )
    df['quantity'] = df['quantity'].fillna(0)


    # Unit price - fill with median by product
    price_medians = df.groupby('product')['unit_price'].median()
    df['unit_price'] = df.apply(
        lambda row: price_medians[row['product']] if pd.isna(row['unit_price']) else row['unit_price'],
        axis=1
    )

    # Region - fill with most common region by customer
    common_regions = df.groupby('customer_name')['region'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')
    df['region'] = df.apply(
        lambda row: common_regions[row['customer_name']] if pd.isna(row['region']) else row['region'],
        axis=1
    )


    # Order date - fill with most recent date
    df['order_date'] = pd.to_datetime(df['order_date'])
    most_recent_date = df['order_date'].max()
    df['order_date'] = df['order_date'].fillna(most_recent_date)

    # 2. Remove duplicates
    print("\n2. Removing duplicates...")
    initial_rows = len(df)
    df = df.drop_duplicates()
    removed = initial_rows - len(df)
    print(f"Removed {removed} duplicate rows")

    # 3. Create derived columns
    print("\n3. Creating derived columns...")
    # Total price
    df['total_price'] = df['quantity'] * df['unit_price']

    # Order month
    df['order_month'] = df['order_date'].dt.to_period('M')

    # Product category (simple grouping)
    product_categories = {
        'Laptop': 'Electronics',
        'Tablet': 'Electronics',
        'Phone': 'Electronics',
        'Monitor': 'Accessories'
    }
    df['product_category'] = df['product'].map(product_categories)

    # 4. Data type conversions
    print("\n4. Converting data types...")
    df['quantity'] = df['quantity'].astype(int)
    df['order_date'] = pd.to_datetime(df['order_date'])

    # 5. Add data quality flags
    print("\n5. Adding data quality flags...")
    df['has_missing_data'] = df.isnull().any(axis=1)

    return df

# Apply transformations
print("=== TRANSFORMING RAW DATA ===")
raw_data = clean_data(raw_data)

print("\n=== TRANSFORMING INCREMENTAL DATA ===")
incremental_data = clean_data(incremental_data)

# Create the 'transformed' directory if it doesn't exist
if not os.path.exists('transformed'):
    os.makedirs('transformed')

# Save transformed data
raw_data.to_csv('transformed/transformed_full.csv', index=False)
incremental_data.to_csv('transformed/transformed_incremental.csv', index=False)

print("\nTransformation complete. Files saved to transformed/ directory")

# Show final results
print("\nFinal Transformed Data Samples:")
print("\nRaw Data:")
display(raw_data.head())
print("\nIncremental Data:")
display(incremental_data.head())

=== TRANSFORMING RAW DATA ===

1. Handling missing values...

2. Removing duplicates...
Removed 1 duplicate rows

3. Creating derived columns...

4. Converting data types...

5. Adding data quality flags...

=== TRANSFORMING INCREMENTAL DATA ===

1. Handling missing values...

2. Removing duplicates...
Removed 0 duplicate rows

3. Creating derived columns...

4. Converting data types...

5. Adding data quality flags...

Transformation complete. Files saved to transformed/ directory

Final Transformed Data Samples:

Raw Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,order_month,product_category,has_missing_data
0,1,Diana,Tablet,2,500.0,2024-01-20,South,1000.0,2024-01,Electronics,False
1,2,Eve,Laptop,2,250.0,2024-04-29,North,500.0,2024-04,Electronics,False
2,3,Charlie,Laptop,2,250.0,2024-01-08,West,500.0,2024-01,Electronics,False
3,4,Eve,Laptop,2,750.0,2024-01-07,West,1500.0,2024-01,Electronics,False
4,5,Eve,Tablet,3,500.0,2024-03-07,South,1500.0,2024-03,Electronics,False



Incremental Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,order_month,product_category,has_missing_data
0,101,Alice,Laptop,1,900.0,2024-05-09,Central,1350.0,2024-05,Electronics,False
1,102,Unknown,Laptop,1,300.0,2024-05-07,Central,300.0,2024-05,Electronics,False
2,103,Unknown,Laptop,1,600.0,2024-05-04,Central,600.0,2024-05,Electronics,False
3,104,Unknown,Tablet,1,300.0,2024-05-26,Central,450.0,2024-05,Electronics,False
4,105,Heidi,Tablet,2,600.0,2024-05-21,North,1200.0,2024-05,Electronics,False
